# Word Embedding

**Non Contextual Word embedding** (или векторное представление слов) - техника в NLP, когда с помощью unsupervised (чаще всего) алгоритма слову или фразе из словаря сопоставляется вещественный вектор фиксированной размерности. Этот вектор (чаще всего) тем или иным образом характеризует семантику слова. Вектор может зависеть от контекста слова (и быть чувствительным к омонимии), а может и не зависеть. 

### Word2Vec

[Word2vec](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) - один из первых методов векторного представления слов,  ставших популярным. 

Два подхода:
- Continues Bag of Words (CBOW)
- Skip gram

![Источник - https://arxiv.org/pdf/1301.3781.pdf ](img/w2v.png)

Вероятность слова в контексте можно интерпретировать как:

$$p(w_o|w_I) = \frac{\exp(v′_{w_O}^T \cdot v_{w_I})}{\sum_{w=1}^{W}\exp(v′_{w_i}^T \cdot v_{w_I})}$$

Для оптимизации используется или negative sampling или hierarchical softmax.

# FastText

[fastText](https://fasttext.cc/) - библиотека от Facebook.
- большое количество предтренированныйх моделей для разных языков
- может сопоставлять вектора для слов вне словаря

Принцип работы - слово делится на N-граммы. Каждой N-грамме сопоставляется вектор, для получения векторного представления всего слова вектора N-грамм суммируются. 

**привет** $\rightarrow$ **&lt;пр**, **при**, **рив**, **иве**, **вет**, **ет&gt;**. 

Дальше идея таже. 

### GloVe

[GloVe](https://nlp.stanford.edu/pubs/glove.pdf) - имплементация через минимизацию функционала:

$$J = \sum_{i,j=1}^V f(X_{ij}) (w_i \cdot \tilde{w_j} + b_i + \tilde{b_j} - \log X_{ij})$$

где, $X_{ij}$ - матрица взаимовстречаемости слов (сколько раз слово $i$ имело в контексте слова $j$). 

$f(x)= \begin{cases}
    (\frac{x}{x_{max}})^\alpha, & \text{если }  x<x_{max}.\\
    1, & \text{иначе}.
  \end{cases}
$

### Пример обучения Word2Vec с помощью [gensim](https://github.com/RaRe-Technologies/gensim)


Каждая строчка в архиве - тройка
> **category** \\t **headline** \\t **text**

Где:
- category - категоря новости
- headline - заголовок
- text - текст новости


#### Обработка данных

In [2]:
import gzip
import re

import gensim

from gensim.models import Word2Vec

from dataclasses import dataclass
from typing import Iterator, List

@dataclass
class Text:
    label: str
    title: str
    text: str

# Чтение файла данных
def read_texts(fn: str="data/news.txt.gz") -> Iterator[Text]:
    with gzip.open(fn, "rt", encoding="utf-8") as f:
        for line in f:
            yield Text(*line.strip().split("\t"))
                    
# Разбиение текста на слова                 
def tokenize_text(text: str) -> List[str]:
    text = text.lower()
    words = re.findall(r'\b\w+\b', text.lower())
    return words

# Текст без знаков припенания (нужен для gensim)
def normalize_text(text: str) -> str:
    return ' '.join(tokenize_text(text))

#### Обучение word2vec

In [3]:
## Обучение word2vec
# каждый текст - набор слов через пробел
sentences = [tokenize_text(text.text) for text in read_texts()]

# обучаем w2v
w2v = Word2Vec(sentences)

# сохраняем модель
w2v.wv.save_word2vec_format('w2v_vectors.bin')
# пример
w2v.wv.most_similar("новости")

[('тасс', 0.807668924331665),
 ('интерфакс', 0.7883888483047485),
 ('риа', 0.7843784689903259),
 ('агентство', 0.7689063549041748),
 ('итар', 0.750734806060791),
 ('источник', 0.7471779584884644),
 ('известия', 0.7186808586120605),
 ('спорт', 0.7142754793167114),
 ('коммерсантъ', 0.706731379032135),
 ('reuters', 0.7004824280738831)]